In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# 数据准备
本例训练集、测试集均从train.csv中取，各10k。

In [2]:
import csv

def read_ad_click_data(n, offset=0): # offset是偏移量，即从第几行开始读
    X_dict, y =[],[]
    with open('train.csv','r') as csvfile: # 本例训练集、测试集均从train.csv中取
        reader = csv.DictReader(csvfile)
        # 跳过前offset行
        for i in range(offset): # 若offset=0,该语句不做任何处理
            next(reader)
        # 逐行读取数据，删除每行的几个字段，即删除这几列特征
        i = 0
        for row in reader:
            i += 1
            y.append(int(row['click']))
            del row['click'],row['id'],row['hour'],row['device_id'],row['device_ip'] 
            X_dict.append(dict(row))
            if i >=n:
                break
    return X_dict,y

In [17]:
n = 10000
X_dict_train, y_train = read_ad_click_data(n)

In [18]:
len(X_dict_train)
len(y_train)

10000

10000

In [19]:
X_dict_train[0]
y_train[0]

{'C1': '1005',
 'C14': '15706',
 'C15': '320',
 'C16': '50',
 'C17': '1722',
 'C18': '0',
 'C19': '35',
 'C20': '-1',
 'C21': '79',
 'app_category': '07d7df22',
 'app_domain': '7801e8d9',
 'app_id': 'ecad2386',
 'banner_pos': '0',
 'device_conn_type': '2',
 'device_model': '44956a24',
 'device_type': '1',
 'site_category': '28905ebd',
 'site_domain': 'f3845767',
 'site_id': '1fbe01fe'}

0

## One-hot Encoding 一位有效编码
将字典里的（特征：特征值）转换为向量

- Pandas.get_dummies()  将string 转换为integers

- Sklearn.preprocessing.OneHotEncoder()  不可以直接处理string，需要先转换为integers

- Sklearn.feature_extraction.DictVectorizer 将字典类型 转换为向量

In [20]:
from sklearn.feature_extraction import DictVectorizer
dict_one_hot_encoder = DictVectorizer(sparse=False)
X_train = dict_one_hot_encoder.fit_transform(X_dict_train)
print(len(X_train[0]))

2820


我们将原来的19维 转换成了 2820维binary特征

### 同上，构造测试集

In [21]:
X_dict_test,y_test = read_ad_click_data(n,n) # 取10k行，跳过前10k行

X_test = dict_one_hot_encoder.transform(X_dict_test)

In [22]:
len(X_test[0])
len(X_dict_test)

2820

10000

# 训练
接下来我们采用grid search 技巧训练一个决策树模型。此处做示范，仅对max_depth调优。

**注意，分类的度量应该用AUC of ROC，因为数据是失衡的，只有1706/10000的样本是click**

In [24]:
from collections import Counter
Counter(y_train)

Counter({0: 8294, 1: 1706})

In [25]:
from sklearn.tree import DecisionTreeClassifier
parameters = {'max_depth': [3,10,None]}
decision_tree = DecisionTreeClassifier(criterion='gini',min_samples_split=30)

In [26]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(decision_tree,parameters, n_jobs=-1, cv=3, scoring='roc_auc')

In [27]:
import timeit
start_time = timeit.default_timer()
grid_search.fit(X_train,y_train)
print('--- %0.3fs seconds ---' % (timeit.default_timer()-start_time))
print(grid_search.best_params_)

GridSearchCV(cv=3, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=30, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 10, None]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring='roc_auc', verbose=0)

--- 26.025s seconds ---
{'max_depth': 10}


## 用最优模型预测测试集：

In [28]:
decision_tree_best = grid_search.best_estimator_
pos_prob = decision_tree_best.predict_proba(X_test)[:,1]

In [29]:
from sklearn.metrics import roc_auc_score # input是y_test 和概率值
print('The ROC AUC on testing set is: {0:.3f}'.format(roc_auc_score(y_test, pos_prob)))

The ROC AUC on testing set is: 0.671


AUC 为0.671，似乎并不是很好。但广告点击行为涉及很多人为因素，其预测本身就很困难。

回顾以下，决策树是在训练集上，通过贪婪算法找每一步的最佳分裂点。但这容易导致过拟合，因为这些最佳点可能只限于训练集中最佳。庆幸的是，随机森林可以纠正这个缺陷，并给出性能更优的树模型

## 随机森林
集成，bagging (bootstrap aggregating)，能很有效地**克服过拟合**。

n个训练集，每个训练集训练出一个独立的分类模型，这些模型‘投票’决出最终的判定。其中每个训练集里的样本，由原始数据集，有放回地随机抽取。

Tree Bagging 能降低决策树模型的高方差问题，所以常优于单棵树。

然而，在某些情况下，其一个或多个特征是强有力的指标，单个树主要是根据这些特征构造的，因而变得高度相关。聚合多个相关树不会有多大差别。为了使每棵树变得不相关，在每个节点寻找最佳分裂点时，随机取几个特征作子集来考虑。现在，**每个训练集的features也不一致**，各树据此训练，保证了更多的多样性和更好的性能。随机森林是，features也bagging（随机获得）的另类tree bagging 模型。

In [38]:
# 同样只对max_depth调参
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=100,criterion='gini',min_samples_split=30,
                                      n_jobs=-1)
grid_search = GridSearchCV(random_forest, parameters,n_jobs=-1,cv=3,scoring='roc_auc')

grid_search.fit(X_train,y_train)
print(grid_search.best_params_)

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=30, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=False,
            random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 10, None]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring='roc_auc', verbose=0)

{'max_depth': None}


In [39]:
random_forest_best = grid_search.best_estimator_
pos_prob = random_forest_best.predict_proba(X_test)[:,1]
print('The AUC of ROC is: {0:.3f}'.format(roc_auc_score(y_test,pos_prob)))

The AUC of ROC is: 0.714


性能有所提升

另有3个重要参数也值得调优，也能提高模型性能：

* max_features: 每次寻找最佳分裂点使考虑的特征数。
    
    1. 惯用的，对于m维的数据集，便取 √m，通过max_features='sqrt'执行；
    2. 其他选项还有'log2'；
    3. 原始特征数的20%-50%。 
    
    
* n_estimators: 
    
    集成几颗树。一般来说，树越多越好，但计算量也会增大。通常设为100、200、500等


* min_samples_split: 

    在一个节点进一步分裂所需的最小样本数。若取值太小，容易过拟合；太大，则欠拟合。10、30、50。
    